In [ ]:
!pip install split-folders

In [2]:
import feather
import pandas as pd
import numpy as np
from numpy import argmax
import matplotlib.pyplot as plt

import wave, os, glob
%matplotlib inline

import librosa
import librosa.display
import IPython.display
import random
import warnings
import os
from PIL import Image
import pathlib
import csv
# sklearn Preprocessing
from sklearn.model_selection import train_test_split
#Keras
import keras
import warnings
warnings.filterwarnings('ignore')
from keras import layers
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add
from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from google.colab import drive


In [ ]:
!pwd
!ls

In [ ]:
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/'CMPT 340 Project'/audio_and_txt_files


In [5]:
path_to_audio_files = []
for filename in glob.glob(os.path.join('', '*.wav')):
    path_to_audio_files.append(filename)
audio_files_data = pd.DataFrame(path_to_audio_files, columns = ['audio_file'])

In [ ]:
# uncomment if you want to drop files with freq < 44100

to_drop = []
audio_files_data['audio_file']
count=0
for file_name in audio_files_data['audio_file']:
  with wave.open(file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        if(frame_rate != 44100):
          to_drop.append(file_name)

for file_name in to_drop:
      audio_files_data = audio_files_data[audio_files_data.audio_file != file_name]

In [ ]:
unique_vals = []
for i in audio_files_data['audio_file'].array:
  unique_vals.append(i[0:3])

In [ ]:
diagnosis =  '101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192  193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226'.split()

In [ ]:
for d in diagnosis:# make folders acc to diagnosis number
    pathlib.Path(f'/content/gdrive/MyDrive/CMPT 340 Project/image_data/{d}').mkdir(parents=True, exist_ok=True) 

In [ ]:
for filename in audio_files_data.audio_file:
        print(filename)
        y, sr = librosa.load(filename, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'/content/gdrive/MyDrive/CMPT 340 Project/image_data/{filename[:3]}/{filename[:-4]}.png')
        plt.clf()

In [ ]:
splitfolders.ratio('/content/gdrive/MyDrive/CMPT 340 Project/image_data', output="/content/gdrive/MyDrive/CMPT 340 Project/data", seed=1337, ratio=(.8, .2))

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True)

test_datagen = ImageDataGenerator(
        rescale=1./255, 
        shear_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True)

In [ ]:
training_set = train_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/CMPT 340 Project/data/train',
        target_size=(64, 64),
        batch_size=1,
        class_mode='categorical',
        shuffle = False)

test_set = test_datagen.flow_from_directory(
        '/content/gdrive/MyDrive/CMPT 340 Project/data/val',
        target_size=(64, 64),
        batch_size=1,
        class_mode='categorical',
        shuffle = False )

In [ ]:
model = Sequential()
input_shape=(64, 64, 3)#1st hidden layer
model.add(Conv2D(32, (3, 3), strides=(2, 2), input_shape=input_shape))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))#2nd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))#3rd hidden layer
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(AveragePooling2D((2, 2), strides=(2,2)))
model.add(Activation('relu'))#Flatten
model.add(Flatten())
model.add(Dropout(rate=0.5))#Add fully connected layer.
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))#Output layer
model.add(Dense(126))
model.add(Activation('softmax'))
model.summary()

In [ ]:
epochs = 255
batch_size = 8
learning_rate = 0.01
decay_rate = learning_rate / epochs
momentum = 0.9
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
model.fit(
        training_set,
        steps_per_epoch=31,
        epochs=21,
        validation_data=test_set,
        validation_steps=165)

In [ ]:
model.evaluate_generator(generator=test_set, steps=50)